In [ ]:
import jsonlines

In [ ]:
def get_friends(_id):
    with jsonlines.open('sampled_vk_dataset_10000.jl') as people:
        for obj in people:
            if obj.get('id') == _id:
                return obj.get('friends')[:]
    return -1

# SOME GARBAGE (TRYING THINGS)

In [ ]:
from sklearn import datasets
from sklearn.semi_supervised import LabelPropagation
import numpy as np
label_prop_model = LabelPropagation()
iris = datasets.load_iris()
rng = np.random.RandomState(42)
random_unlabeled_points = rng.rand(len(iris.target)) < 0.3
labels = np.copy(iris.target)
labels[random_unlabeled_points] = -1
label_prop_model.fit(iris.data, labels)

In [ ]:
import os
import jsonlines
path = './seeds_graphs_ids'
files = os.listdir(path)
seeds = []
for _file in files:
    seeds.append(int(_file[9:-4]))

In [ ]:
male = 0
female = 0
single = 0
married = 0
higher = 0
secondary = 0
none = 0
with jsonlines.open('sampled_vk_dataset_10000.jl') as profiles:
    for item in profiles:
        if item.get('id') in seeds:
            if item.get('gender') == 'male':
                male += 1
            elif item.get('gender') == 'female':
                female += 1
            if item.get('status') == 'single':
                single += 1
            elif item.get('status') == 'married':
                married += 1
            if item.get('education') == 'high':
                higher += 1
            elif item.get('education') == 'middle':
                secondary += 1
            else:
                none += 1

In [ ]:
print('male', male)
print('female', female)
print('single', single)
print('married', married)
print('higher', higher)
print('secondary', secondary)
print('none', none)

# END OF GARBAGE

# Подграфы

In [ ]:
import jsonlines
import os, sys 
from tqdm import tqdm
import numpy as np
import re

In [ ]:
seeds_path = "./distribs3seed"
seeds = sorted(os.listdir(seeds_path))

In [ ]:
path = "./distribs4"
files = sorted(os.listdir(path))

In [ ]:
max_len = 0
for _file in files:
    with open(path + '/' + _file) as distribution:
        line = distribution.readline()
        vec = re.findall(r'(\d+)', line)
        max_len = max(max_len, len(vec))

In [ ]:
x = []
for _file in ordered_seeds:
    with open(path + '/' + str(_file)) as distribution:
        line = distribution.readline()
        vec = re.findall(r'(\d+)', line)
        while len(vec) < 2:
            vec.append('0')
        vec = list(map(int,vec))
        x.append(vec)

In [ ]:
for i in range(len(x)):
    x[i] = list(map(int, x[i]))

In [ ]:
seeds = list(map(int, files))

In [ ]:
y_gender_ = [' '] * len(seeds)
y_age_ = [0] * len(seeds)
y_status_ = [' '] * len(seeds)
y_edu_ = [' '] * len(seeds)

with jsonlines.open('sampled_vk_dataset_10000.jl') as profiles:
    for item in profiles:
        if int(item.get('id')) in seeds:
            y_gender[seeds.index(item.get('id'))] = item.get('gender')
            y_age[seeds.index(item.get('id'))] = int(item.get('age'))
            y_status[seeds.index(item.get('id'))] = item.get('status')
            y_edu[seeds.index(item.get('id'))] = item.get('education')

In [ ]:
for i in range(len(x)):
    if sum(x[i]) != 0:
        x[i] = np.array(x[i]) / np.linalg.norm(x[i])
    else:
        x[i] = np.array(x[i])
x = np.array(x)

In [ ]:
_x = []
_y = []
colors = []
for i in range(len(x)):
    _x.append(x[i][0])
    _y.append(x[i][1])
    if (y_edu[i] == 'high'):
        colors.append('green')
    elif (y_edu[i] == 'middle'):
        colors.append('yellow')
    else:
        colors.append('red')

In [ ]:
from matplotlib import pyplot as plt
plt.scatter(_x, _y, color=colors, s=5)
plt.savefig('education_log.png', dpi=200)

In [ ]:
plt.gcf().clear()

In [ ]:
#SVM
from sklearn.model_selection import cross_val_score
from sklearn import svm

In [ ]:
#SVM edu
clf = svm.SVC()
scores = cross_val_score(clf, x_gender, y_gender, cv=5, scoring='f1_micro')
np.average(scores)

In [ ]:
#Logistic regression
from sklearn import linear_model

In [ ]:
#Logistic regression edu
_clf = linear_model.LogisticRegression()
scores = cross_val_score(_clf, x, y_gender, cv=5, scoring='f1_macro')
np.average(scores)

In [ ]:
#GaussianNB
from sklearn.naive_bayes import GaussianNB

In [ ]:
#GaussianNB edu
model = GaussianNB()
scores = cross_val_score(model, x, y_edu, cv=5, scoring='f1_macro')
np.average(scores)

In [ ]:
#Decision tree
from sklearn import tree

In [ ]:
#Decision tree edu
clf = tree.DecisionTreeClassifier()
scores = cross_val_score(clf, x, y_edu, cv=5, scoring='f1_macro')
np.average(scores)

In [ ]:
#KNeighbours
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#KNeighbours edu
neigh = KNeighborsClassifier(n_neighbors=3)
scores = cross_val_score(neigh, x, y_edu, cv=5, scoring='f1_macro')
np.average(scores)

In [ ]:
#XGBoost
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
y_edu = np.array(y_edu)
x = np.array(list(map(np.array, x)))

In [ ]:
#XGBoost Gender
model = XGBClassifier()
scores = cross_val_score(model, x_conc_age, y_age, cv=5, scoring='f1_macro')
np.average(scores)

# Всякие комбинации

# Возраст как классификация

In [ ]:
#распределения возрастов друзей
path = "./distribs3"
seeds_age = sorted(os.listdir(path))
seeds_age = list(map(int, seeds_age))

In [ ]:
#вытаскиваем друзей
ordered_seeds = []
ordered_friends = []
with jsonlines.open('sampled_vk_dataset_10000.jl') as profiles:
    for item in profiles:
        if int(item.get('id')) in seeds_age:
            ordered_seeds.append(int(item.get('id')))
            ordered_friends.append(item.get('friends'))

In [ ]:
#пространство Y
y_age = [0] * len(ordered_seeds)

with jsonlines.open('sampled_vk_dataset_10000.jl') as profiles:
    for item in profiles:
        if int(item.get('id')) in ordered_seeds:
            y_age[ordered_seeds.index(int(item.get('id')))] = (int(item.get('age')) + 5 - int(item.get('age')) % 5) // 5

In [ ]:
#пространство X
x_age = [[0] * 22 for x in range(len(ordered_seeds))]

with jsonlines.open('sampled_vk_dataset_10000.jl') as profiles:
    for item in profiles:
        for i in range(len(ordered_friends)):
            if item.get('id') in list(ordered_friends[i]):
                if item['age']:
                    _class = (int(item.get('age')) + 5 - int(item.get('age')) % 5) // 5
                    x_age[i][_class - 1] += 1

In [ ]:
#нормировка
for i in range(len(x_age)):
    if sum(x_age[i]) != 0:
        x_age[i] = np.array(x_age[i]) / np.linalg.norm(np.array(x_age[i]), ord=2)
        
x_age = np.array(x_age)
y_age = np.array(y_age)

In [ ]:
#сохранение
from joblib import dump

dump(x_age, "XAgeClasses")
dump(y_age, "YAgeClasses")

In [ ]:
#центроиды
from sklearn.neighbors import NearestCentroid
clf_age = NearestCentroid()
clf_age.fit(x_age, y_age)
centroids_age = clf_age.centroids_
dict_age = dict(zip(clf_age.classes_, centroids_age))

In [ ]:
x_full_age = []
y_full_age = []
for i in range(len(x)):
    for j in range(3, 14):
        x_full_age.append(np.concatenate((x[i], dict_age[j])))
        if y_age[i] == j:
            y_full_age.append(1)
        else:
            y_full_age.append(0)
x_full_age = np.array(x_full_age)
y_full_age = np.array(y_full_age)

In [ ]:
#конкатенация
x_conc_age = []
for i in range(len(x)):
    x_conc_age.append(np.concatenate((x_age[i], x[i])))
x_conc_age = np.array(x_conc_age)

In [ ]:
#распределения признаков друзей
seeds_friends_lst = []
seeds_lst = []
seeds_path = "./distribs3"
seeds = sorted(os.listdir(seeds_path))
seeds = list(map(int, seeds))

In [ ]:
true_seeds = []
true_seeds_friends = []
with jsonlines.open('sampled_vk_dataset_10000.jl') as profiles:
    for item in profiles:
        if int(item.get('id')) in seeds:
            true_seeds.append(int(item.get('id')))
            true_seeds_friends.append(item.get('friends'))
seeds_lst = true_seeds[:]
seeds_friends_lst = true_seeds_friends[:]

In [ ]:
#пространство Y (признаки сидов)
y_gender = [' '] * len(seeds_lst)
y_age = [0] * len(seeds_lst)
y_status = [' '] * len(seeds_lst)
y_edu = [' '] * len(seeds_lst)

with jsonlines.open('sampled_vk_dataset_10000.jl') as profiles:
    for item in profiles:
        if int(item.get('id')) in seeds_lst:
            y_gender[seeds_lst.index(int(item.get('id')))] = item.get('gender')
            y_age[seeds_lst.index(int(item.get('id')))] = int(item.get('age'))
            y_status[seeds_lst.index(int(item.get('id')))] = item.get('status')
            y_edu[seeds_lst.index(int(item.get('id')))] = item.get('education')

In [ ]:
#пространство X (распределения признаков друзей)
x_gender = [[0, 0] for x in range(len(seeds))]
x_status = [[0, 0] for x in range(len(seeds))]
x_age = [[0] * 110 for x in range(len(seeds))]
x_edu = [[0, 0, 0] for x in range(len(seeds))]

with jsonlines.open('sampled_vk_dataset_10000.jl') as profiles:
    for item in profiles:
        for i in range(len(seeds_friends_lst)):
            if item.get('id') in list(seeds_friends_lst[i]):
                if item.get('gender') == 'male':
                    x_gender[i][0] += 1
                elif item.get('gender') == 'female':
                    x_gender[i][1] += 1
                if item.get('status') == 'single':
                    x_status[i][0] += 1
                elif item.get('status') == 'married':
                    x_status[i][1] += 1
                if item['age']:
                    x_age[i][int(item.get('age'))] += 1
                if item.get('education') == 'high':
                    x_edu[i][0] += 1
                elif item.get('education') == 'middle':
                    x_edu[i][1] += 1
                else:
                    x_edu[i][2] += 1

In [ ]:
#нормировка
for i in range(len(x_age)):
    if sum(x_gender[i]) != 0:
        x_gender[i] = np.array(x_gender[i]) / np.linalg.norm(np.array(x_gender[i]), ord=2)
    if sum(x_status[i]) != 0:
        x_status[i] = np.array(x_status[i]) / np.linalg.norm(np.array(x_status[i]), ord=2)
    if sum(x_age[i]) != 0:
        x_age[i] = np.array(x_age[i]) / np.linalg.norm(np.array(x_age[i]), ord=2)
    if sum(x_edu[i]) != 0:
        x_edu[i] = np.array(x_edu[i]) / np.linalg.norm(np.array(x_edu[i]), ord=2)
        
x_gender = np.array(x_gender)
x_status = np.array(x_status)
x_age = np.array(x_age)

In [ ]:
y_gender = np.array(y_gender)
y_age = np.array(y_age)
y_status = np.array(y_status)

In [ ]:
x_edu = np.array(x_edu)
y_edu = np.array(y_edu)

In [ ]:
from joblib import dump

dump(x_gender, "XGender")
dump(x_status, "XStatus")
dump(x_edu, "XEducation")

dump(y_gender, "YGender")
dump(y_status, "YStatus")
dump(y_edu, "YEducation")

dump(x_age, "XAge")
dump(y_age, "YAge")

In [ ]:
#центроиды
from sklearn.neighbors import NearestCentroid

In [ ]:
clf_gender = NearestCentroid()
clf_gender.fit(x_gender, y_gender)
centroids_gender = clf_gender.centroids_
dict_gender = dict(zip(clf_gender.classes_, centroids_gender))

clf_status = NearestCentroid()
clf_status.fit(x_status, y_status)
centroids_status = clf_status.centroids_
dict_status = dict(zip(clf_status.classes_, centroids_status))

clf_edu = NearestCentroid()
clf_edu.fit(x_edu, y_edu)
centroids_edu = clf_edu.centroids_
dict_edu = dict(zip(clf_edu.classes_, centroids_edu))

In [ ]:
x_full_status = []
y_full_status = []
for i in range(len(x)):
    x_full_status.append(np.concatenate((x[i], dict_status['single'])))
    x_full_status.append(np.concatenate((x[i], dict_status['married'])))
    if y_status[i] == 'single':
        y_full_status.append(1)
        y_full_status.append(0)
    elif y_status[i] == 'married':
        y_full_status.append(0)
        y_full_status.append(1)

In [ ]:
for i in range(len(x_cont_age)):
    if sum(x_cont_age[i]) != 0:
        x_cont_age[i] = np.array(x_cont_age[i]) / np.linalg.norm(np.array(x_cont_age[i]), ord=2)

In [ ]:
x_cont_age = []
for i in range(len(x)):
    x_cont_age.append(np.concatenate((x_age[i], x[i])))

# Label Propagation

In [ ]:
from sklearn.semi_supervised import LabelPropagation
from sklearn.model_selection import cross_val_score

In [ ]:
label_prop_model = LabelPropagation(n_neighbors=7, max_iter=1)
scores = cross_val_score(label_prop_model, true_x_gender, true_y_gender, cv=5, scoring='f1_micro')
np.average(scores)

In [ ]:
#SVM
from sklearn.model_selection import cross_val_score
from sklearn import svm

In [ ]:
x_gender

In [ ]:
#SVM gender
clf = svm.SVC()
scores = cross_val_score(clf, x_conc_age, y_age, cv=5, scoring='f1_macro')
np.average(scores)

In [ ]:
scores = cross_val_score(clf, true_x_gender, true_y_gender, cv=5, scoring='f1_macro')
np.average(scores)

In [ ]:
#SVM status
clf = svm.SVC()
scores = cross_val_score(clf, x_status, y_status, cv=5, scoring='f1_micro')
np.average(scores)

In [ ]:
from sklearn import linear_model

In [ ]:
#Logistic regression gender
clf = linear_model.LogisticRegression()
scores = cross_val_score(clf, x_conc_age, y_age, cv=5, scoring='f1_macro')
np.average(scores)

In [ ]:
scores = cross_val_score(clf, x_gender, y_gender, cv=10, scoring='f1_macro')
np.average(scores)

In [ ]:
#Logistic regression status
clf = linear_model.LogisticRegression()
scores = cross_val_score(clf, x_status, y_status, cv=5, scoring='f1_micro')
np.average(scores)

In [ ]:
#GaussianNB
from sklearn.naive_bayes import GaussianNB

In [ ]:
#GaussianNB gender
clf = GaussianNB()
scores = cross_val_score(clf, x_conc_age, y_age, cv=5, scoring='f1_macro')
np.average(scores)

In [ ]:
#GaussianNB status
clf = GaussianNB()
scores = cross_val_score(clf, x_status, y_status, cv=5, scoring='f1_micro')
np.average(scores)

In [ ]:
#Decision tree
from sklearn import tree

In [ ]:
#Decision tree gender
clf = tree.DecisionTreeClassifier()
scores = cross_val_score(clf, x_conc_age, y_age, cv=5, scoring='f1_macro')
np.average(scores)

In [ ]:
#Decision tree status
clf = tree.DecisionTreeClassifier()
scores = cross_val_score(clf, x_status, y_status, cv=5, scoring='f1_micro')
np.average(scores)

In [ ]:
#KNeighbours
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#KNeighbours gender
neigh = KNeighborsClassifier(n_neighbors=3)
scores = cross_val_score(neigh, x_conc_age, y_age, cv=5, scoring='f1_macro')
np.average(scores)

In [ ]:
#KNeighbours status
neigh = KNeighborsClassifier(n_neighbors=3)
scores = cross_val_score(neigh, x_status, y_status, cv=5, scoring='f1_macro')
np.average(scores)

In [ ]:
#Linear regression age MAE
reg = linear_model.LinearRegression()
scores = cross_val_score(reg, x_cont_age, y_age, cv=5, scoring='neg_median_absolute_error')
np.average(scores)

In [ ]:
#Ridge regression age MAE
from sklearn.linear_model import Ridge
clf = Ridge(alpha=1.0)
scores = cross_val_score(clf, x_cont_age, y_age, cv=5, scoring='neg_median_absolute_error')
np.average(scores)

In [ ]:
true_y_status = []
true_x_status = []
for i in range(len(x_status)):
    if not x_status[i][0] != x_status[i][0]:
        true_y_status.append(y_status[i])
        true_x_status.append(x_status[i])
true_y_status = np.array(true_y_status)
true_x_status = np.array(true_x_status)

In [ ]:
true_y_age = []
true_x_age = []
for i in range(len(x_age)):
    if not x_age[i][0] != x_age[i][0]:
        true_y_age.append(y_age[i])
        true_x_age.append(x_age[i])
true_y_age = np.array(true_y_age)
true_x_age = np.array(true_x_age)

In [ ]:
true_y_gender = []
true_x_gender = []
for i in range(len(x_gender)):
    if not x_gender[i][0] != x_gender[i][0]:
        true_y_gender.append(y_gender[i])
        true_x_gender.append(x_gender[i])
true_y_gender = np.array(true_y_gender)
true_x_gender = np.array(true_x_gender)

# ОБРАЗОВАНИЕ

In [ ]:
seeds_edu = []
with jsonlines.open('sampled_vk_dataset_10000.txt.seeds.txt') as seeds:
    for seed in seeds:
        seeds_edu.append(seed)

In [ ]:
seeds_friends_edu = []
true_seeds_edu = []
with jsonlines.open('sampled_vk_dataset_10000.jl') as profiles:
    for item in profiles:
        if item.get('id') in seeds_edu:
            true_seeds_edu.append(item.get('id'))
            seeds_friends_edu.append(item.get('friends'))

In [ ]:
#Пространство Y для образования
y_edu = [' '] * len(true_seeds_edu)

with jsonlines.open('sampled_vk_dataset_10000.jl') as profiles:
    for item in profiles:
        if item.get('id') in true_seeds_edu:
            y_edu[true_seeds_edu.index(item.get('id'))] = item.get('education')

In [ ]:
#Пространство X для образования
x_edu = [[0, 0, 0] for x in range(len(true_seeds_edu))]

with jsonlines.open('sampled_vk_dataset_10000.jl') as profiles:
    for item in profiles:
        for i in range(len(seeds_friends_edu)):
            if item.get('id') in list(seeds_friends_edu[i]):
                if item.get('education') == 'high':
                    x_edu[i][0] += 1
                elif item.get('education') == 'middle':
                    x_edu[i][1] += 1
                else:
                    x_edu[i][2] += 1

In [ ]:
#нормировка
import numpy as np
x_edu = np.array([np.array(vec) / np.linalg.norm(np.array(vec), ord=2) for vec in x_edu])

In [ ]:
y_edu = np.array(y_edu)

In [ ]:
true_y_edu = []
true_x_edu = []
for i in range(len(x_edu)):
    if not x_edu[i][0] != x_edu[i][0]:
        true_y_edu.append(y_edu[i])
        true_x_edu.append(x_edu[i])
true_y_edu = np.array(true_y_edu)
true_x_edu = np.array(true_x_edu)

In [ ]:
#SVM образование
from sklearn.model_selection import cross_val_score
from sklearn import svm

In [ ]:
num = 0
for i in range(len(true_y_gender)):
    if true_y_gender[i] == 'male':
        num += 1
len(true_y_age)

In [ ]:
clf = svm.SVC()
scores = cross_val_score(clf, x_edu, y_edu, cv=5, scoring='f1_micro')
np.average(scores)

In [ ]:
#Logistic regression образование
from sklearn import linear_model

In [ ]:
clf = linear_model.LogisticRegression()
scores = cross_val_score(clf, x_edu, y_edu, cv=5, scoring='f1_micro')
np.average(scores)

In [ ]:
#GaussianNB образоание
from sklearn.naive_bayes import GaussianNB

In [ ]:
clf = GaussianNB()
scores = cross_val_score(clf, x_edu, y_edu, cv=5, scoring='f1_micro')
np.average(scores)

In [ ]:
#Decision tree образование
from sklearn import tree

In [ ]:
clf = tree.DecisionTreeClassifier()
scores = cross_val_score(clf, x_edu, y_edu, cv=5, scoring='f1_micro')
np.average(scores)

In [ ]:
#KNeighbours обрзование
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)
scores = cross_val_score(neigh, x_edu, y_edu, cv=5, scoring='f1_micro')
np.average(scores)

# XGBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
x_edu = np.array(x_edu)
y_edu = np.array(y_edu)

In [ ]:
model = XGBClassifier()
scores = cross_val_score(model, x_edu, y_edu, cv=5, scoring='f1_micro')
np.average(scores)

In [ ]:
model = XGBClassifier()
scores = cross_val_score(model, true_x_status, true_y_status, cv=5, scoring='f1_macro')
np.average(scores)

In [ ]:
num = 0
for i in range(len(true_y_edu)):
    if true_y_edu[i] == 'none':
        num += 1
num

In [ ]:
def build_connections(_id):
    cur_friends = get_friends(_id)
    
    if cur_friends == -1:
        return -1
    
    connections = {}
    connections[_id] = cur_friends
    with jsonlines.open('sampled_vk_dataset_10000.jl') as people:
        for obj in people:
            if obj.get('id') in cur_friends:
                connections[obj.get('id')] = obj.get('friends')
    return connections

In [ ]:
def get_graph(_id):
    connects = build_connections(_id)
    
    if connects == -1:
        return -1
    
    conn_keys = connects.keys()
    graph = set()
    for key in conn_keys:
        friends_lst = connects[key]
        for friend in friends_lst:
            graph.add((min(key, friend), max(key, friend), 1))
    return graph

In [ ]:
def get_renumbered_graph(_id):
    cur_valid = 1
    renum_ids = {}
    graph = get_graph(_id)
    
    if graph == -1:
        return -1
    
    graph = list(graph)
    outfile = './seeds_graphs/graph_vk_' + str(_id) + '.txt' 
    with open(outfile, 'w') as g:
        for edge in graph:
            e_lst = list(edge)
            if not(e_lst[0] in renum_ids.keys()):
                renum_ids[e_lst[0]] = cur_valid
                cur_valid += 1
            if not(e_lst[1] in renum_ids.keys()):
                renum_ids[e_lst[1]] = cur_valid
                cur_valid += 1
            print(' '.join(map(str, [renum_ids[e_lst[0]], renum_ids[e_lst[1]], 1])), file=g)

In [ ]:
from multiprocessing.dummy import Pool as ThreadPool

pool = ThreadPool(4)

In [ ]:
ids_lst = []
with open('sampled_vk_dataset_10000.txt.seeds.txt') as ids:
    cur_num = 1
    for _id in ids:
        if cur_num >= 6451:
            ids_lst.append(int(_id))
        cur_num += 1

In [ ]:
pool.map(get_renumbered_graph, ids_lst)

# Label Propagation

In [ ]:
import os, sys
import re
import networkx as nx
import jsonlines
from tqdm import tqdm
from labelprop import LabelProp

In [ ]:
path = './boneless/'
files = os.listdir(path)

In [ ]:
import random
random.shuffle(files)
files = files[:100]

In [ ]:
seeds_lp = [int(item[9:-4]) for item in files]

In [ ]:
seeds_props = [0] * len(files)
with jsonlines.open('sampled_vk_dataset_10000.jl') as profiles:
    for item in profiles:
        if item.get('id') in seeds_lp:
            if item.get('education') == 'none':
                seeds_props[seeds_lp.index(item.get('id'))] = 1
            elif item.get('education') == 'middle':
                seeds_props[seeds_lp.index(item.get('id'))] = 2
            elif item.get('education') == 'high':
                seeds_props[seeds_lp.index(item.get('id'))] = 3

In [ ]:
seeds_props.count(3)

In [ ]:
def lp_graph(path):
    G = nx.Graph()
    
    with open(path) as _graph:
        for edge in _graph:
            G.add_edge(int(edge.split()[0]), int(edge.split()[1]))
        
    with jsonlines.open('sampled_vk_dataset_10000.jl') as profiles:
        for item in profiles:
            if int(item.get('id')) in G.nodes and int(item.get('id')) != int(path[20:-4]):
                if item.get('education') == 'none':
                    G.nodes[int(item.get('id'))]['edu'] = 1
                elif item.get('education') == 'middle':
                    G.nodes[int(item.get('id'))]['edu'] = 2
                elif item.get('education') == 'high':
                    G.nodes[int(item.get('id'))]['edu'] = 3
                else:
                    G.nodes[int(item.get('id'))]['status'] = 0
                    
    for node in G.nodes:
        if 'edu' not in G.nodes[node].keys():
            G.nodes[node]['edu'] = 0
            
    graph_lp = []
    for node in G.nodes:
        graph_item = []
        graph_item.append(node)
        graph_item.append(G.nodes[node]['edu'])
        neighs = [[neigh, 1] for neigh in G.neighbors(node)]
        graph_item.append(neighs)
        graph_lp.append(graph_item)
        
    lp = LabelProp()
    lp.load_data_from_mem(graph_lp)
    
    ans = lp.run(0.00001, 2, clean_result=True) 
    
    for node in ans:
        if node[0] == int(path[20:-4]):
            return node[1]

In [ ]:
predicted = []
for _file in tqdm(files):
    predicted.append(lp_graph(path + _file))

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(seeds_props, predicted, average='weighted')

In [ ]:
seeds_props.count(2)

In [ ]:
G = nx.Graph()

In [ ]:
with open(path + files) as _graph:
    for edge in _graph:
        G.add_edge(int(edge.split()[0]), int(edge.split()[1]))

In [ ]:
with jsonlines.open('sampled_vk_dataset_10000.jl') as profiles:
    for item in profiles:
        if int(item.get('id')) in G.nodes:
            if item.get('gender') == 'male':
                G.nodes[int(item.get('id'))]['gender'] = 1
            elif item.get('gender') == 'female':
                G.nodes[int(item.get('id'))]['gender'] = 2
            else:
                G.nodes[int(item.get('id'))]['gender'] = 0
            if item.get('status') == 'single':
                G.nodes[int(item.get('id'))]['status'] = 1
            elif item.get('status') == 'married':
                G.nodes[int(item.get('id'))]['status'] = 2
            else:
                G.nodes[int(item.get('id'))]['status'] = 0
            if item.get('education') == 'none':
                G.nodes[int(item.get('id'))]['edu'] = 1
            elif item.get('education') == 'middle':
                G.nodes[int(item.get('id'))]['edu'] = 2
            elif item.get('education') == 'high':
                G.nodes[int(item.get('id'))]['edu'] = 3
            else:
                G.nodes[int(item.get('id'))]['edu'] = 0

In [ ]:
for node in tqdm(G.nodes):
    if 'gender' not in G.nodes[node].keys():
        G.nodes[node]['gender'] = 0
    if 'status' not in G.nodes[node].keys():
        G.nodes[node]['status'] = 0
    if 'edu' not in G.nodes[node].keys():
        G.nodes[node]['edu'] = 0

In [ ]:
graph_lp = []
for node in tqdm(G.nodes):
    graph_item = []
    graph_item.append(node)
    graph_item.append(G.nodes[node]['gender'])
    neighs = [[neigh, 1] for neigh in G.neighbors(node)]
    graph_item.append(neighs)
    graph_lp.append(graph_item)

In [ ]:
lp = LabelProp()
lp.load_data_from_mem(graph_lp)

In [ ]:
ans = lp.run(0.00001, 2, clean_result=True) 

In [ ]:
ans

In [ ]:
path = "./seeds_graphs_profiles/"
dirs = os.listdir(path)

In [ ]:
def pickline(line):
    i = 0
    with open('sampled_vk_dataset_10000.txt.seeds.txt') as _file:
        for item in _file:
            if i == line - 1:
                return str(item)
            i += 1

In [ ]:
for _dir in dirs:
    if int(_dir[9:-12]) <= 3007:
        _id = pickline(int(_dir[9:-12]))
        src = './seeds_graphs_profiles/' + _dir
        #print(src)
        dst = re.sub('\d+', _id[:-1], _dir)
        #print(dst)
        dst = './seeds_graphs_profiles/' + dst
        #print(dst)
        os.rename(src, dst)

In [ ]:
dirs = os.listdir(path)

In [ ]:
with jsonlines.open('seeds_distributions.jl', mode='w') as outfile:
    for _dir in dirs:
        with open('./seeds_graphs_profiles/' + _dir + '/undir-3') as distrib:
            opcl = []
            for _line in distrib:
                opcl.append(float(_line.split()[1]))
            opcl = (opcl[0], opcl[1])
            desc = {'id': int(_dir[9:-12]), '(open, closed)': opcl}
            outfile.write(desc)

In [ ]:
people = []
x = []
with jsonlines.open('seeds_distributions.jl') as distribs:
    for item in distribs:
        people.append(item.get('id'))
        x.append(item.get('(open, closed)'))

In [ ]:
y_gender = []
y_age = []
y_education = []
y_status = []
with jsonlines.open('sampled_vk_dataset_10000.jl') as items:
    for item in items:
        if item.get('id') in people:
            y_gender.append(item.get('gender'))
            y_age.append(item.get('age'))
            y_education.append(item.get('education'))
            y_status.append(item.get('status'))

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model

#gender Logistic regression
X_train, X_test, y_train, y_test = train_test_split(
    x, y_gender, test_size=0.3, random_state=0)

clf = linear_model.LogisticRegression().fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
#education Logistic regression
X_train, X_test, y_train, y_test = train_test_split(
    x, y_education, test_size=0.3, random_state=0)

clf = linear_model.LogisticRegression().fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
#status Logistic regression
X_train, X_test, y_train, y_test = train_test_split(
    x, y_status, test_size=0.3, random_state=0)

clf = linear_model.LogisticRegression().fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
#age Logistic regression
X_train, X_test, y_train, y_test = train_test_split(
    x, y_age, test_size=0.3, random_state=0)

clf = linear_model.LogisticRegression().fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
import numpy as np
y_gender = np.array(y_gender)
y_age = np.array(y_age)
y_education = np.array(y_education)
y_status = np.array(y_status)
x = np.array(x)

In [ ]:
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    x, y_gender, test_size=0.3, random_state=42)

In [ ]:
#gender SVM
clf = SVC()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
#education SVM
clf = SVC()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
#status SVM
clf = SVC()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
#age SVM
clf = SVC()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
#gender GaussianNB
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
#education GaussianNB
clf = GaussianNB()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
#status GaussianNB
clf = GaussianNB()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
#age GaussianNB
clf = GaussianNB()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
from sklearn.metrics import precision_score
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(criterion="entropy", random_state=1)
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
precision_score(clf.predict(X_train), y_train, average='macro')

# Trying stuff... again (garbage)

In [ ]:
import jsonlines

with jsonlines.open("tru_bois.jl", mode="w") as out:
    with jsonlines.open("sampled_vk_dataset_10000.jl", mode="r") as info:
        for i in info:
            if i["id"] not in people:
                continue
                
            accepted_friends = []
            for f in i["friends"]:
                if f in people:
                    accepted_friends.append(f)
            #if len(accepted_friends) > 0:
            out.write({'id': i['id'], 'friends': accepted_friends, 
                       'gender': i['gender'],
                      'age': i['age'], 'status': i['status'],
                      'education': i['education']})


In [ ]:
edu = set()
stat = set()
with jsonlines.open("sampled_vk_dataset_10000.jl", mode="r") as info:
    for item in info:
        edu.add(item['education'])
        stat.add(item['status'])
print(edu)
print(stat)

In [ ]:
seed_friends = []
with jsonlines.open("sampled_vk_dataset_10000.jl", mode="r") as ppl:
    for item in ppl:
        if item.get('id') in people:
            seed_friends.append(item.get('friends'))

In [ ]:
x_gender = [[0, 0] for x in range(len(people))]
x_status = [[0, 0] for x in range(len(people))]
x_edu = [[0, 0, 0] for x in range(len(people))]
x_age = [[0] * 110 for x in range(len(people))]
bound = len(seed_friends)

with jsonlines.open("sampled_vk_dataset_10000.jl", mode="r") as ppl:
    for item in ppl:
        for i in range(bound):
            if item.get('id') in seed_friends[i]:
                if item.get('gender') == 'male':
                    x_gender[i][0] += 1
                elif item.get('gender') == 'female':
                    x_gender[i][1] += 1
                if item.get('status') == 'single':
                    x_status[i][0] += 1
                elif item.get('status') == 'married':
                    x_status[i][1] += 1
                if item.get('education') == 'high':
                    x_edu[i][0] += 1
                elif item.get('education') == 'middle':
                    x_edu[i][1] += 1
                elif item.get('education') == 'none':
                    x_edu[i][2] += 1
                if item['age']:
                    x_age[i][int(item['age'])] += 1

In [ ]:
x_gender[255]

In [ ]:
with jsonlines.open('sampled_vk_dataset_10000.jl', mode='r') as _people:
    files = [0, 0, 0]
    files[0] = jsonlines.open('sampled_vk_dataset_part1.jl', mode='w')
    files[1] = jsonlines.open('sampled_vk_dataset_part2.jl', mode='w')
    files[2] = jsonlines.open('sampled_vk_dataset_part3.jl', mode='w')
    for i, man in enumerate(_people):
        files[i % 3].write(man)
    for file in files:
        file.close()


In [ ]:
import random